[Jeder kann coden](../../abstract/Contents.de.ipynb) / [Programmieren & TicTacToe](../../Programming_And_TicTacToe.de.ipynb) / [Objektorientierte Programmierung](../../Objectoriented_Programming.de.ipynb) / [Objektorientierte Programmierung in C#](CSharp_Basics.de.ipynb)

# Events

<table border="0">
  <tr>
    <td>
        <img src="Events.jpeg">
    </td>
    <td rowspan="2">
        <a href="https://miro.com/app/board/o9J_lOJi2o0=/?moveToWidget=3458764554347680798&cot=14"><img src="Radar_Lambda_n_Delegate.jpg"></a>
    </td>
  </tr>
  <tr>
    <td>
      <a href="https://learn.microsoft.com/de-de/dotnet/csharp/programming-guide/events/">Events (Microsoft Docs)</a><br>
      <a href="https://learn.microsoft.com/de-de/dotnet/csharp/programming-guide/delegates/">Delegates (Microsoft Docs)</a><br>
      <a href="https://learn.microsoft.com/de-de/dotnet/csharp/language-reference/operators/lambda-expressions">Lambda-Ausdrücke (Microsoft Docs)</a><br>
      <a href="https://www.c-sharpcorner.com/UploadFile/1e050f/events-in-C-Sharp/">Events in C# – C# Corner</a><br>
      <a href="https://code-maze.com/csharp-events/">Understanding Events in C# (Code Maze)</a><br>
      <a href="https://www.tutorialsteacher.com/csharp/csharp-event">C# Event Tutorial – TutorialsTeacher</a><br>
      <a href="https://www.dotnetperls.com/event">C# Event Examples – DotNetPerls</a><br>
      <a href="https://refactoring.guru/design-patterns/observer/csharp/example">Observer Pattern in C# – Refactoring Guru</a><br>
      <a href="https://learn.microsoft.com/de-de/dotnet/api/system.eventhandler-1">System.EventHandler&lt;T&gt; API-Dokumentation</a><br>
      <a href="https://stackoverflow.com/questions/2406668/difference-between-event-and-delegate">StackOverflow: Unterschied zwischen Event und Delegate</a>
    </td>
  </tr>
</table>

Ein **Event** ist ein Mechanismus, mit dem ein Objekt **Benachrichtigungen** über bestimmte Änderungen oder Aktionen an interessierte Abonnenten (Observer) senden kann. Es basiert auf dem **Publisher/Subscriber-Prinzip**.

**Events nutzen Delegaten als Typ** – also Methoden-Signaturen, die aufgerufen werden, wenn das Event ausgelöst wird.

### Grundstruktur eines Events

In [1]:
public class Alarm
{
    public event Action AlarmAusgeloest;

    public void Ausloesen()
    {
        Console.WriteLine("Alarm wird ausgelöst!");
        AlarmAusgeloest?.Invoke(); // Aufruf aller registrierten Methoden
    }
}

#### Wichtige Merkmale:

* `event` schränkt den Zugriff ein (z. B. keine direkten Zuweisungen von außen)
* Nur `+=` (Abonnieren) und `-=` (Abmelden) sind erlaubt von außen

### Warum `event` statt nur `Delegate`?

Ohne `event` könnten andere Objekte die gesamte Delegatenliste **überschreiben** – mit `event` wird das verhindert:

```csharp
public Action SomethingHappened; // gefährlich – kann überschrieben werden
public event Action SaferEvent;  // nur += und -= von außen erlaubt
```

### Verwendung mit Lambdas

Wenn du Lambdas kennst, kannst du elegant Event-Handler registrieren:

In [2]:
var alarm = new Alarm();
alarm.AlarmAusgeloest += () => Console.WriteLine("Alarm wurde gehört!");
alarm.Ausloesen();

Alarm wird ausgelöst!
Alarm wurde gehört!


### Beispiel mit eigenen EventArgs

Für komplexere Daten kannst du eigene EventArgs-Klassen nutzen:

In [3]:
public class TemperatureEventArgs : EventArgs
{
    public double Temperatur { get; }

    public TemperatureEventArgs(double temperatur)
    {
        Temperatur = temperatur;
    }
}

public class Thermometer
{
    public event EventHandler<TemperatureEventArgs> TemperaturGeaendert;

    public void NeueTemperatur(double temperatur)
    {
        TemperaturGeaendert?.Invoke(this, new TemperatureEventArgs(temperatur));
    }
}

Verwendung:

In [6]:
Thermometer thermometer = new Thermometer(); // Erstellen einer Instanz des Thermometers
thermometer.NeueTemperatur(25.0); // Nutzen der Methode, um eine neue Temperatur zu setzen

// Registrieren eines Ereignishandlers für Temperaturänderungen
thermometer.TemperaturGeaendert += (sender, args) =>
{
    Console.WriteLine($"Neue Temperatur: {args.Temperatur} °C");
};

thermometer.NeueTemperatur(30.0); // Setzen einer neuen Temperatur, was das Ereignis auslöst

Neue Temperatur: 30 °C


### 💡 Best Practices

| Thema        | Empfehlung                                                                |
| ------------ | ------------------------------------------------------------------------- |
| EventHandler | Nutze `EventHandler` oder `EventHandler<T>` für eigene Events             |
| Nullprüfung  | Verwende `?.Invoke()` oder sichere Aufrufe                                |
| Kapselung    | Mach Events `public`, aber nur `private`-set für Auslöser                 |
| Benennung    | Verwende das Verb in der Vergangenheitsform: `SomethingHappened`          |
| Abmeldung    | Denke daran, Lambdas können nicht wieder abgemeldet werden (kein Handle!) |

### 🧨 Schwachstelle: Anonyme Handler

Du kannst einen Event-Handler mit Lambda registrieren, aber **nicht wieder entfernen**, wenn du kein Handle hast:

```csharp
// Diese Zeile lässt sich nicht wieder entfernen!
obj.EinEvent += (s, e) => Console.WriteLine("..."); 
```

Alternative:

```csharp
EventHandler handler = (s, e) => Console.WriteLine("...");
obj.EinEvent += handler;
// später:
obj.EinEvent -= handler;
```

### Custom Accessors (fortgeschritten)

Du kannst eigene `add` / `remove` Accessoren definieren:

```csharp
private EventHandler myEvent;

public event EventHandler MyEvent
{
    add
    {
        Console.WriteLine("Abonniert!");
        myEvent += value;
    }
    remove
    {
        Console.WriteLine("Abgemeldet!");
        myEvent -= value;
    }
}
```

### Events vs Callbacks

| Aspekt      | Event                | Callback                       |
| ----------- | -------------------- | ------------------------------ |
| Teilnehmer  | Viele Abonnenten     | Genau eine Methode             |
| Bindung     | Dynamisch via `+=`   | Direkt als Parameter übergeben |
| Architektur | Publisher/Subscriber | Direkt gekoppelt               |

### Zusammenfassung

* Events sind eine sichere, strukturierte Art, Delegaten zu veröffentlichen
* Verwende `event`-Schlüsselwort, um Zugriff einzuschränken
* Nutze `EventHandler` / `EventHandler<T>` für standardkonforme Signaturen
* Lambdas sind praktisch – aber ohne Referenz nicht wieder abzumelden
* Achte auf Speicherlecks durch nicht entfernte Handler (z. B. bei statischen Events!)

## Cheatsheet für Events

<a href="https://miro.com/app/board/o9J_lOJi2o0=/?moveToWidget=3458764525304733582&cot=14"><img src="CheatsheetEvents.jpg"></a>

## Vertiefung

### 1. Multicast-Delegaten & Aufrufreihenfolge

Ein Event in C# ist ein **Multicast-Delegat**. Das heißt: Mehrere Methoden können sich registrieren und werden **in der Reihenfolge des `+=`** aufgerufen.

```csharp
myEvent += MethodeA;
myEvent += MethodeB;
```

Wenn `myEvent` ausgelöst wird, geschieht dies in der Reihenfolge:

* `MethodeA`
* `MethodeB`

Falls eine Methode eine **Exception** wirft, werden die **restlichen nicht aufgerufen**, es sei denn du fängst sie manuell ab:

```csharp
foreach (var handler in myEvent.GetInvocationList())
{
    try { handler.DynamicInvoke(); }
    catch (Exception ex) { Console.WriteLine($"Fehler: {ex.Message}"); }
}
```

### 2. Memory Leaks durch Event-Handler

Ein typischer Fehler: Du abonnierst ein Event, aber meldest dich nie wieder ab. Wenn der Publisher dann **lange lebt (z. B. statisch oder Singleton)**, wird der Subscriber **niemals vom GC freigegeben**.

Beispiel:

```csharp
publisher.MyEvent += subscriber.HandleSomething;
// subscriber wird nie vom GC freigegeben, solange publisher lebt!
```

**Lösungen:**

* `-=` bei `Dispose`
* `WeakEventManager` bei WPF
* `IDisposable`-Pattern im Subscriber

### 3. Schwache Events in WPF / MVVM

WPF verwendet **`WeakEventManager`**, um Speicherlecks zu vermeiden:

```csharp
WeakEventManager<Publisher, EventArgs>.AddHandler(publisher, nameof(Publisher.MyEvent), HandlerMethode);
```

Vorteil: der Publisher hält keinen **starken Verweis**, sodass der Subscriber gesammelt werden kann.

### 4. Testbarkeit von Events

Events sind **schwerer testbar**, weil:

* Sie keine Rückgabewerte haben
* Sie keinen direkten Einfluss auf das Objekt selbst haben

**Lösungen:**

* Event-Trigger beobachten via Mocks
* Oder: explizit `InvokeEventXYZ()` im Test aufrufen und Resultat prüfen

### 5. Events und Interfaces

Interfaces können Events enthalten:

```csharp
public interface IDownloadTask
{
    event EventHandler<ProgressEventArgs> ProgressChanged;
}
```

Das ist nützlich für lose Kopplung und Substitution (z. B. in MVVM).

### 6. Events verketten (Chaining)

Du kannst Events an Events binden:

```csharp
obj1.Changed += obj2.OnChanged;
obj2.Changed += obj3.OnChanged;
// Vorsicht mit Event Loops!
```

**Tipp**: Schütze dich vor Endlosschleifen durch einfache Logik:

```csharp
if (!alreadyHandled)
{
    alreadyHandled = true;
    RaiseChanged();
}
```

### 7. Thread-Sicherheit beim Event-Aufruf

Events sind nicht automatisch **Thread-safe**. Ein häufiger Pattern:

```csharp
var handler = MyEvent;
if (handler != null)
    handler(this, EventArgs.Empty);
```

Oder ab C# 6+:

```csharp
MyEvent?.Invoke(this, EventArgs.Empty);
```

Wenn das Event von einem anderen Thread auf `null` gesetzt wird, kann es sonst zu `NullReferenceException` kommen.

### 8. Events serialisierbar machen

Events selbst sind **nicht serialisierbar** (weil Delegaten keine Daten sind). Du musst explizit **vom Event ausgelöste Zustände speichern** – nicht das Event selbst.

### 9. Custom Event Pattern für Fehlerbehandlung oder Logging

Du kannst eigene Event-Zwischenschichten bauen, z. B. für:

* Logging bei jeder Registrierung
* Fehlerbehandlung
* Synchronisierung

Beispiel:

```csharp
private EventHandler myEvent;
public event EventHandler MyEvent
{
    add {
        Console.WriteLine("Abonnent hinzugefügt");
        myEvent += value;
    }
    remove {
        Console.WriteLine("Abonnent entfernt");
        myEvent -= value;
    }
}
```

### 10. Events über Netzwerke (SignalR, PubSub etc.)

In verteilten Systemen (Blazor, WPF+Backend, Microservices) nutzt man **Events als Konzept**, aber technisch brauchst du:

* **SignalR/Websockets** für UI
* **Message Broker** (z. B. NATS, RabbitMQ) für Backend

Ein Event in C# wird dann z. B. nach außen als Message serialisiert und verschickt.

### Merksätze

| Thema       | Merksatz                                                |
| ----------- | ------------------------------------------------------- |
| Lebensdauer | Wer sich anmeldet, sollte sich abmelden                 |
| Zugriff     | Nur innerhalb der Klasse darf `Invoke` verwendet werden |
| Lambdas     | Nur mit Handle wieder abmeldbar                         |
| Standard    | Verwende `EventHandler<T>` mit `EventArgs`              |
| Testbarkeit | Lieber explizite Trigger oder Mock-Zugriffe nutzen      |